In [ ]:
from ccb import io
import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
benchmark_name = "converted"


data = []
for task in io.task_iterator(benchmark_name):
    loss = task.eval_loss
    if isinstance(loss, type):
        loss = loss()
    task_dict = {
        'name': task.dataset_name,
        'img size': task.patch_size,
        'loss': str(loss),
        'label type': repr(task.label_type),
    }
    data.append(task_dict)


df = pd.DataFrame.from_records(data)
pd.set_option('max_colwidth', 400)


df